# Colab inference for ZFTurbo's [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training/)


<font size=1>*made by [jarredou](https://github.com/jarredou)</font>  
[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Q5Q811R5YI)

In [ ]:
#@markdown #GDrive connection
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import base64
#@markdown # Install

%cd /content
!git clone -b colab-inference https://github.com/jarredou/Music-Source-Separation-Training

#requirements fix by santilli_
req_text = """
mutagen==1.47.0
ml_collections==1.1.0
numpy>=1.26.0
pandas==2.2.2
scipy
tqdm
segmentation_models_pytorch==0.3.3
timm
audiomentations
pedalboard
omegaconf
beartype
rotary_embedding_torch==0.3.5
einops
# librosa==0.11.0
demucs #==4.0.0
# transformers==4.35.0
torchmetrics==0.11.4
spafe==0.3.2
protobuf
torch_audiomentations
asteroid==0.7.0
auraloss
torchseg
"""

with open("Music-Source-Separation-Training/requirements.txt", "w") as f:
    f.write(req_text)

!mkdir '/content/Music-Source-Separation-Training/ckpts'

print('Installing the dependencies... This will take a few minutes')
!pip install -r 'Music-Source-Separation-Training/requirements.txt' &> /dev/null

print('Installation is done !')

In [ ]:
%cd '/content/Music-Source-Separation-Training/'
import os
import torch
import yaml
from urllib.parse import quote

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)


def tuple_constructor(loader, node):
    # Load the sequence of values from the YAML node
    values = loader.construct_sequence(node)
    # Return a tuple constructed from the sequence
    return tuple(values)

# Register the constructor with PyYAML
yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple',
tuple_constructor)



def conf_edit(config_path, chunk_size, overlap):
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)

    # handle cases where 'use_amp' is missing from config:
    if 'use_amp' not in data.keys():
      data['training']['use_amp'] = True

    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    if data['inference']['batch_size'] == 1:
      data['inference']['batch_size'] = 2

    print("Using custom overlap and chunk_size values:")
    print(f"overlap = {data['inference']['num_overlap']}")
    print(f"chunk_size = {data['audio']['chunk_size']}")
    print(f"batch_size = {data['inference']['batch_size']}")

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)

def download_file(url):
    # Encode the URL to handle spaces and special characters
    encoded_url = quote(url, safe=':/')

    path = 'ckpts'
    os.makedirs(path, exist_ok=True)
    filename = os.path.basename(encoded_url)
    file_path = os.path.join(path, filename)

    if os.path.exists(file_path):
        print(f"File '{filename}' already exists at '{path}'.")
        return

    try:
        response = torch.hub.download_url_to_file(encoded_url, file_path)
        print(f"File '{filename}' downloaded successfully")
    except Exception as e:
        print(f"Error downloading file '{filename}' from '{url}': {e}")





#@markdown # Separation
#@markdown #### Separation config:
input_folder = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
model = 'VOCALS-MelBand-Roformer (by KimberleyJSN)' #@param ['VOCALS-MelBand-Roformer (by KimberleyJSN)', 'VOCALS-BS-Roformer_1297 (by viperx)', 'VOCALS-BS-Roformer_1296 (by viperx)', 'VOCALS-InstVocHQ', 'KARAOKE-MelBand-Roformer (by becruily)', 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)',  '6STEMS-BS-Roformer-SW', '4STEMS-SCNet_XL_MUSDB18 (by ZFTurbo)', '4STEMS-SCNet_Large (by starrytong)', '4STEMS-BS-Roformer_MUSDB18 (by ZFTurbo)', '4STEMS-SCNet_MUSDB18 (by starrytong)', 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)', 'CINEMATIC-BandIt_Plus (by kwatcharasupat)', 'CINEMATIC-BandIt_v2 multi (by kwatcharasupat)', 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)', 'DE-REVERB-MDX23C (by aufr33 & jarredou)', 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by anvuew)', 'DE-REVERB-MelBand-Roformer mono 20.4029 (by anvuew)', 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)', 'DENOISE-MelBand-Roformer-1 (by aufr33)', 'DENOISE-MelBand-Roformer-2 (by aufr33)', 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by aufr33)', 'GUITAR-MelBand-Roformer (by becruily)']
extract_instrumental = True #@param {type:"boolean"}
export_format = 'flac PCM_16' #@param ['wav FLOAT', 'flac PCM_16', 'flac PCM_24']
use_tta = False #@param {type:"boolean"}
#@markdown ---
#@markdown *Roformers custom config:*
overlap = 2 #@param {type:"slider", min:2, max:40, step:1}
chunk_size = "485100" #@param [88200, 352800, 485100, 588800] {allow-input: true}
chunk_size = int(chunk_size)

if export_format.startswith('flac'):
    flac_file = True
    pcm_type = export_format.split(' ')[1]
else:
    flac_file = False
    pcm_type = None

if model == 'VOCALS-InstVocHQ':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_vocals_mdx23c.yaml'
    start_check_point = 'ckpts/model_vocals_mdx23c_sdr_10.17.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/config_vocals_mdx23c.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_mdx23c_sdr_10.17.ckpt')

elif model == 'VOCALS-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_vocals_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/config_vocals_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/mel_band_roformer_vocals_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'INST-MelBand-Roformer (by Becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_instrumental_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_instrumental_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/config_instrumental_becruily.yaml')
    download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/mel_band_roformer_instrumental_becruily.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
    start_check_point = 'ckpts/MelBandRoformer.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
    download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1297 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml')
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1296 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.ckpt'
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt')
    download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_368_sdr_12.9628.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_karaoke_becruily.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_becruily.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-karaoke/resolve/main/mel_band_roformer_karaoke_becruily.ckpt')
    download_file('https://huggingface.co/becruily/mel-band-roformer-karaoke/resolve/main/config_karaoke_becruily.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_mel_band_roformer_karaoke.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-melroformer-model/resolve/main/config_mel_band_roformer_karaoke.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_crowd.yaml'
    start_check_point = 'ckpts/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/model_mel_band_roformer_crowd.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'CINEMATIC-BandIt_Plus (by kwatcharasupat)':
    model_type = 'bandit'
    config_path = 'ckpts/config_dnr_bandit_bsrnn_multi_mus64.yaml'
    start_check_point = 'ckpts/model_bandit_plus_dnr_sdr_11.47.chpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/config_dnr_bandit_bsrnn_multi_mus64.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/model_bandit_plus_dnr_sdr_11.47.chpt')

elif model == 'CINEMATIC-BandIt_v2 multi (by kwatcharasupat)':
    model_type = 'bandit_v2'
    config_path = 'ckpts/config_dnr_bandit_v2_mus64.yaml'
    start_check_point = 'ckpts/checkpoint-multi_state_dict.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/refs/heads/main/configs/config_dnr_bandit_v2_mus64.yaml')
    download_file('https://huggingface.co/jarredou/banditv2_state_dicts_only/resolve/main/checkpoint-multi_state_dict.ckpt')

elif model == 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml'
    start_check_point = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt'
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt')
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml')

elif model == '6STEMS-BS-Roformer-SW':
    model_type = 'bs_roformer'
    config_path = 'ckpts/BS-Rofo-SW-Fixed.ckpt'
    start_check_point = 'ckpts/BS-Rofo-SW-Fixed.ckpt'
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.yaml')
    download_file('https://huggingface.co/jarredou/BS-ROFO-SW-Fixed/resolve/main/BS-Rofo-SW-Fixed.ckpt')

elif model == '4STEMS-SCNet_XL_MUSDB18 (by ZFTurbo)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet_xl.yaml'
    start_check_point = 'ckpts/model_scnet_ep_54_sdr_9.8051.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.13/config_musdb18_scnet_xl.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.13/model_scnet_ep_54_sdr_9.8051.ckpt')

elif model == '4STEMS-SCNet_Large (by starrytong)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet_large_starrytong.yaml'
    start_check_point = 'ckpts/SCNet-large_starrytong_fixed.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.9/config_musdb18_scnet_large_starrytong.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.9/SCNet-large_starrytong_fixed.ckpt')

elif model == '4STEMS-SCNet_MUSDB18 (by starrytong)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet.yaml'
    start_check_point = 'ckpts/scnet_checkpoint_musdb18.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/config_musdb18_scnet.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/scnet_checkpoint_musdb18.ckpt')

elif model == '4STEMS-BS-Roformer_MUSDB18 (by ZFTurbo)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_bs_roformer_384_8_2_485100.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_17_sdr_9.6568.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.12/config_bs_roformer_384_8_2_485100.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.12/model_bs_roformer_ep_17_sdr_9.6568.ckpt')

elif model == 'DE-REVERB-MDX23C (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_dereverb_mdx23c.yaml'
    start_check_point = 'ckpts/dereverb_mdx23c_sdr_6.9096.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/dereverb_mdx23c_sdr_6.9096.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/config_dereverb_mdx23c.yaml')

elif model == 'DE-REVERB-MelBand-Roformer aggr./v2/19.1729 (by anvuew)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/dereverb_mel_band_roformer_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew.yaml')

elif model == 'DE-REVERB-MelBand-Roformer mono 20.4029 (by anvuew)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/dereverb_mel_band_roformer_anvuew.yaml'
    start_check_point = 'ckpts/dereverb_mel_band_roformer_mono_anvuew_sdr_20.4029.ckpt'
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_mono_anvuew_sdr_20.4029.ckpt')
    download_file('https://huggingface.co/anvuew/dereverb_mel_band_roformer/resolve/main/dereverb_mel_band_roformer_anvuew.yaml')

elif model == 'DE-REVERB-Echo-MelBand-Roformer (by Sucial)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_dereverb-echo_mel_band_roformer.yaml'
    start_check_point = 'ckpts/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt'
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt')
    download_file('https://huggingface.co/Sucial/Dereverb-Echo_Mel_Band_Roformer/resolve/main/config_dereverb-echo_mel_band_roformer.yaml')

elif model == 'DENOISE-MelBand-Roformer-1 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'DENOISE-MelBand-Roformer-2 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-Male Female-BS-RoFormer Male Female Beta 7_2889 (by aufr33)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_chorus_male_female_bs_roformer.yaml'
    start_check_point = 'ckpts/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt'
    download_file('https://huggingface.co/RareSirMix/AIModelRehosting/resolve/main/bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt')
    download_file('https://huggingface.co/Sucial/Chorus_Male_Female_BS_Roformer/resolve/main/config_chorus_male_female_bs_roformer.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'PHANTOM-CENTER-HTDemucs (by wesleyr36)':
    model_type = 'htdemucs'
    config_path = 'ckpts/config_htdemucs_similarity.yaml'
    start_check_point = 'ckpts/model_htdemucs_ep_21_sdr_13.6970.ckpt'
    download_file('https://huggingface.co/jarredou/HTDemucs_Similarity_Extractor_by_wesleyr36/resolve/main/model_htdemucs_ep_21_sdr_13.6970.ckpt')
    download_file('https://huggingface.co/jarredou/HTDemucs_Similarity_Extractor_by_wesleyr36/resolve/main/config_htdemucs_similarity.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'GUITAR-MelBand-Roformer (by becruily)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_guitar_becruily.yaml'
    start_check_point = 'ckpts/becruily_guitar.ckpt'
    download_file('https://huggingface.co/becruily/mel-band-roformer-guitar/resolve/main/becruily_guitar.ckpt')
    download_file('https://huggingface.co/becruily/mel-band-roformer-guitar/resolve/main/config_guitar_becruily.yaml')
    conf_edit(config_path, chunk_size, overlap)


!python inference.py \
    --model_type {model_type} \
    --config_path '{config_path}' \
    --start_check_point '{start_check_point}' \
    --input_folder '{input_folder}' \
    --store_dir '{output_folder}' \
    {('--extract_instrumental' if extract_instrumental else '')} \
    {('--flac_file' if flac_file else '')} \
    {('--use_tta' if use_tta else '')} \
    {('--pcm_type ' + pcm_type if pcm_type else '')}

In [ ]:
drive.mount("/content/drive", force_remount=True)